<a href="https://www.kaggle.com/code/gizemnalbantarslan/rfm-customer-segmentation?scriptVersionId=113165649" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 

FLO, a shoe company, wants to develop different campaigns for different customer groups.In this study, customers were analyzed and grouped in accordance with the purpose.

RFM analysis method will be used during the study.

# RFM ANALYSIS

* Data Understanding
* Data Preparation
* Calculating RFM Metrics
* Calculating RFM Scores
* Creating & Analysing RFM Segments

**Variables**

* master_id: Unique customer number
* order_channel: Which channel of the shopping platform is used (Android, ios, Desktop, Mobile)
* last_order_channel: The channel where the most recent purchase was made
* first_order_date: Date of the customer's first purchase
* last_order_date: Date of the customer's last purchase
* last_order_date_online: Date of the customer's last purchase on the online platform
* last_order_date_offline: Date of the customer's last purchase on the offline platform
* order_num_total_ever_online: Number of purchases made by the customer on the online platform
* order_num_total_ever_offline: Number of purchases made by the customer on the offline platform
* customer_value_total_ever_offline: The total fee paid by the customer for their offline shopping
* customer_value_total_ever_online: The total fee paid by the customer for their online shopping
* interested_in_categories_12: List of categories the customer has shopped in the last 12 months

 **General Data Structure**

In [2]:
import datetime as dt
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [3]:
df_ = pd.read_csv("../input/flo-rfm-analysis/flo_data_20k.csv")
df = df_.copy()

df.head(10)
df.info()
df.describe().T
df.isnull().values.any()
df.isnull().sum()
df.dtypes
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

(19945, 12)

Omnichannel means that customers shop from both online and offline platforms. New variables were created for the total number of purchases and spending of each customer.Changed the type of variables that express date but have object data type to date.

In [4]:
df["TotalOrderNumber"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total_ever"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
df.head()

date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

The number of customers in the shopping channels, the total number of products purchased and the distribution of total expenditures were examined.

In [5]:
df.groupby("order_channel").agg({"master_id": "count",
                                 "TotalOrderNumber": "sum",
                                 "customer_value_total_ever": "sum"})

df.groupby("master_id").agg({"customer_value_total_ever": "sum"}).sort_values("customer_value_total_ever", ascending=False).head(10)
df.groupby("master_id").agg({"TotalOrderNumber": "sum"}).sort_values("TotalOrderNumber", ascending=False).head(10)

,TotalOrderNumber
master_id,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.000
cba59206-9dd1-11e9-9897-000d3a38a36f,131.000
a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.000
fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.000
329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.000
73fd19aa-9e37-11e9-9897-000d3a38a36f,82.000
44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.000
b27e241a-a901-11e9-a2fc-000d3a38a36f,75.000
d696c654-2633-11ea-8e1c-000d3a38a36f,70.000


All this data preparation process has been functionalized.

In [6]:
def prepare_data(dataframe):
    dataframe["TotalOrderNumber"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total_ever"] = (dataframe["order_num_total_ever_online"] * dataframe["customer_value_total_ever_online"]) + \
                                             (dataframe["order_num_total_ever_offline"] * dataframe["customer_value_total_ever_offline"])

    for col in dataframe.columns:
        if "date" in col:
            dataframe[col] = pd.to_datetime(dataframe[col])

    return dataframe.head()

prepare_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

Customer specific Recency, Frequency and Monetary scores were calculated.

In [7]:
df.head()
df["last_order_date"].max()

today_date = dt.datetime(2021, 6, 1)

rfm = df.groupby("master_id").agg({"last_order_date": lambda last_order_date: (today_date - last_order_date.max()).days,
                                     "TotalOrderNumber": lambda TotalOrderNumber: TotalOrderNumber,
                                     "customer_value_total_ever": lambda customer_value_total_ever: customer_value_total_ever.sum()})

rfm.columns = ["recency", "frequency", "monetary"]
rfm.head()

rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm.head()

rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

rfm["RF_SCORE"] = (rfm["recency_score"].astype(str)) + (rfm["frequency_score"].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.000,1979.330,5,4,4,54
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.000,419.950,1,2,2,12
000be838-85df-11ea-a90b-000d3a38a36f,213,4.000,1436.630,2,3,3,23
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.000,3153.670,5,4,4,54
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.000,7201.710,5,4,5,54


Segment definitions were made for the generated RF scores and scores were converted into segments.

In [8]:
seg_map = {
    r"[1-2][1-2]": "hibernating",
    r"[1-2][3-4]": "at_Risk",
    r"[1-2]5": "cant_loose",
    r"3[1-2]": "about_to_sleep",
    r"33": "need_attention",
    r"[3-4][4-5]": "loyal_customers",
    r"41": "promising",
    r"51": "new_customers",
    r"[4-5][2-3]": "potential_loyalist",
    r"5[4-5]": "champions"
}

rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm.head()

rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean"])

,recency,frequency,monetary
,mean,mean,mean
segment,,,
about_to_sleep,114.032,2.407,481.625
at_Risk,242.329,4.470,1829.340
cant_loose,235.159,10.717,16088.820
champions,17.142,8.965,19499.284
hibernating,247.426,2.391,480.075
loyal_customers,82.558,8.356,10954.101
need_attention,113.037,3.739,1211.415
new_customers,17.976,2.000,344.049


FLO includes a new women's shoe brand. The product prices of the brand it includes are above the general customer preferences. For this reason, it is desired to contact the customers in the profile that will be interested in the promotion of the brand and product sales. Those who shop from their loyal customers (champions, loyal_customers) and women category are the customers to be contacted specifically. The id numbers of these customers were found and saved in the cvs file.

In [9]:
rfm=pd.merge(rfm, df, how='left', on='master_id')
womendf = rfm[["master_id", "segment", "interested_in_categories_12"]]


womendf = womendf.loc[(womendf["interested_in_categories_12"].str.contains("KADIN")) &
                      ((womendf["segment"] == "loyal_customers") | (womendf["segment"] == "champions"))]

womendf[["master_id"]].to_csv("a_target_customer_id.csv")

Nearly 40% discount is planned for Men's and Children's products. It is aimed to specifically target customers who are good customers in the past, but who have not shopped for a long time, who are interested in the categories related to this discount, who should not be lost, those who are asleep and new customers. The ids of the customers in the appropriate profile are saved in the csv file.

In [10]:
boys_discountdf = rfm[["master_id", "segment", "interested_in_categories_12"]]
boys_discountdf


boys_discountdf = boys_discountdf.loc[((boys_discountdf["interested_in_categories_12"].str.contains("COCUK")) |
                           (boys_discountdf["interested_in_categories_12"].str.contains("ERKEK"))) &
                           ((boys_discountdf["segment"] == "hibernating") |
                           (boys_discountdf["segment"] == "cant_loose") |
                           (boys_discountdf["segment"] == "new_customers"))]